## Make sure to run Chunking Series and Durations first. This will not be accurate if those files are not up to date.

In [1]:
import pandas as pd
import numpy as np

In [2]:
scaleColumns = ['EntryFeeAmount', 'TotalPrizeAmount','MaxNumberPlayers', 'MaxEntriesPerUser', 'DistinctUsers', 'NumGames', 'DraftablePlayersInSet','PaidUsersInDraftGroup', 'TopPrize', 'MaxPayoutPosition', 'Duration']
categoricalColumns = ['SportName', 'VariantName', 'Contest_Group']

In [11]:
rawDF = pd.read_csv('data/WorkingData.csv')
rawDF = pd.merge(rawDF, pd.read_csv('data/Durations.csv').set_index('ContestId'), on='ContestId', how='left')

In [9]:
lrDF.head()

,f10A,f10B,f11A,f11B,f1A,f1B,f2A,f2B,f3A,f3B,...,Predf11,Predf1,Predf2,Predf3,Predf4,Predf5,Predf6,Predf7,Predf8,Predf9
ContestId,,,,,,,,,,,,,,,,,,,,,
55826033,402.019102,0.000064,401.847675,0.000064,401.828632,0.000064,inf,-5.387283,inf,-0.482865,...,35678.616189,35678.442450,0.0,0.0,0.000000,21.866193,40698.196480,37326.414033,35851.339144,35695.808116
56610037,186.177344,0.000088,186.146297,0.000088,186.142848,0.000088,inf,-3.909979,inf,-0.347349,...,3336.231229,3336.226084,0.0,0.0,0.000000,114.867891,3654.563416,3385.953549,3341.354319,3336.740434
56645954,30.568268,0.000080,30.566681,0.000080,30.566505,0.000080,inf,-2.158675,inf,-0.191472,...,110.734358,110.734342,0.0,0.0,0.008826,49.817043,111.265956,110.884477,110.750313,110.735949
56751298,0.661184,0.000232,0.661178,0.000232,0.661178,0.000232,inf,-2.392493,inf,-0.215720,...,150.883703,150.883357,0.0,0.0,0.036452,379.785123,187.448641,154.359868,151.229038,150.917906
56776765,345.670006,0.000524,345.655886,0.000524,345.654317,0.000524,inf,-4.322272,inf,-0.381426,...,15249.238253,15249.234401,0.0,0.0,0.007296,13234.808953,15578.977966,15287.219352,15253.081628,15249.619609


In [10]:
lrDF = pd.read_csv('data/LR_Preds.csv').set_index('ContestId').drop(columns='Duration')
kfDF = pd.read_csv('data/KF_Preds.csv').set_index('ContestId').drop(columns='Duration')
# rawDF = pd.merge(rawDF, kfDF, on='ContestId', how='left').set_index('ContestId')

In [30]:
cDF = pd.merge(rawDF, kfDF, on='ContestId', how='left')
cDF = pd.merge(cDF, lrDF, on='ContestId', how='left')
cDF = cDF.set_index('ContestId')

In [36]:
# processedDF.columns

In [61]:
processedDF = cDF.copy(deep=True)
metaColumns = []
kfColumns = []
lrColumns = []
successCol = []
abColumns = []


processedDF['Success'] = (processedDF['MaxNumberPlayers'] == processedDF['Entries'])
successCol.append('Success')

successDF = processedDF[['Success']]

for scol in scaleColumns:
    processedDF[scol+"_Scaled"] = processedDF[scol]/processedDF[scol].max()
    metaColumns.append(scol+"_Scaled")
    
for scol in categoricalColumns:
    for val in processedDF[scol].unique():
        catStr = scol+"_"+val
        processedDF[catStr] = (processedDF[scol]==val)
        metaColumns.append(catStr)
        
metaDF = processedDF[metaColumns]

for col in kfDF.columns:
    if(col[:5]=="Predv"):
        label = col+"_Scaled"
        processedDF[label] = processedDF[col]/processedDF["MaxNumberPlayers"]
        kfColumns.append(label)
    elif(col[0]=='A' or col[0]=='B'):
        abColumns.append(col)

kfPredsDF = processedDF[kfColumns]
        
for col in lrDF.columns:
    if(col[:5]=="Predf"):
        processedDF[label] = processedDF[col]/processedDF["MaxNumberPlayers"]
        lrColumns.append(label)
    elif(col[::-1][0]=='A' or col[::-1][0]=='B'):
        abColumns.append(col)
        
        
lrPredsDF = processedDF[lrColumns]

abDF = processedDF[abColumns]

In [64]:
# lrDF

In [53]:
kfPredsDF.to_csv('data/KF_Results.csv')
lrPredsDF.to_csv('data/LR_Results.csv')
metaDF.to_csv('data/MetaData.csv')
abDF.to_csv('data/AB_Values.csv')

In [54]:
workedData = pd.merge(metaDF, kfPredsDF, on='ContestId', how='left')
workedData = pd.merge(workedData, lrPredsDF, on='ContestId', how='left')
workedData = pd.merge(workedData, abDF, on='ContestId', how='left')
workedData = pd.merge(successDF, workedData, on='ContestId', how='left')

In [55]:
workedData.to_csv('data/WorkedData.csv')